# Test 01

In [1]:

import sys
lib_decomren = "/home/dotamthuc-3090/Projects/ViewSynthesis/DeConRen"
sys.path.insert(0, lib_decomren)

import itertools
import numpy as np
import torch
import decompress_render

In [2]:
pos_np = np.array(tuple(itertools.product([0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3])))
pos = torch.tensor(pos_np, dtype=torch.int64).cuda()
nlevel = torch.tensor([2], dtype=torch.int8).cuda()
morton_code = decompress_render.utils.position_to_mortoncode(pos, nlevel).cpu().numpy()
combi = np.concatenate([pos_np, morton_code], axis=1)[np.argsort(morton_code[:, 0])]

combi

array([[ 0,  0,  0,  0],
       [ 0,  0,  1,  1],
       [ 0,  1,  0,  2],
       [ 0,  1,  1,  3],
       [ 1,  0,  0,  4],
       [ 1,  0,  1,  5],
       [ 1,  1,  0,  6],
       [ 1,  1,  1,  7],
       [ 0,  0,  2,  8],
       [ 0,  0,  3,  9],
       [ 0,  1,  2, 10],
       [ 0,  1,  3, 11],
       [ 1,  0,  2, 12],
       [ 1,  0,  3, 13],
       [ 1,  1,  2, 14],
       [ 1,  1,  3, 15],
       [ 0,  2,  0, 16],
       [ 0,  2,  1, 17],
       [ 0,  3,  0, 18],
       [ 0,  3,  1, 19],
       [ 1,  2,  0, 20],
       [ 1,  2,  1, 21],
       [ 1,  3,  0, 22],
       [ 1,  3,  1, 23],
       [ 0,  2,  2, 24],
       [ 0,  2,  3, 25],
       [ 0,  3,  2, 26],
       [ 0,  3,  3, 27],
       [ 1,  2,  2, 28],
       [ 1,  2,  3, 29],
       [ 1,  3,  2, 30],
       [ 1,  3,  3, 31],
       [ 2,  0,  0, 32],
       [ 2,  0,  1, 33],
       [ 2,  1,  0, 34],
       [ 2,  1,  1, 35],
       [ 3,  0,  0, 36],
       [ 3,  0,  1, 37],
       [ 3,  1,  0, 38],
       [ 3,  1,  1, 39],


In [3]:

nlevel = torch.ones((2), dtype=torch.int8).cuda()
decompress_render.utils.mortoncode_to_position(torch.tensor(combi[:, 3], dtype=torch.int64).cuda(), nlevel)

tensor([[0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
        [0, 0, 2],
        [0, 0, 3],
        [0, 1, 2],
        [0, 1, 3],
        [1, 0, 2],
        [1, 0, 3],
        [1, 1, 2],
        [1, 1, 3],
        [0, 2, 0],
        [0, 2, 1],
        [0, 3, 0],
        [0, 3, 1],
        [1, 2, 0],
        [1, 2, 1],
        [1, 3, 0],
        [1, 3, 1],
        [0, 2, 2],
        [0, 2, 3],
        [0, 3, 2],
        [0, 3, 3],
        [1, 2, 2],
        [1, 2, 3],
        [1, 3, 2],
        [1, 3, 3],
        [2, 0, 0],
        [2, 0, 1],
        [2, 1, 0],
        [2, 1, 1],
        [3, 0, 0],
        [3, 0, 1],
        [3, 1, 0],
        [3, 1, 1],
        [2, 0, 2],
        [2, 0, 3],
        [2, 1, 2],
        [2, 1, 3],
        [3, 0, 2],
        [3, 0, 3],
        [3, 1, 2],
        [3, 1, 3],
        [2, 2, 0],
        [2, 2, 1],
        [2, 3, 0],
        [2, 3, 1],
        [3, 

# Test 02

In [1]:

import sys
lib_decomren = "/home/dotamthuc-3090/Projects/ViewSynthesis/DeConRen"
sys.path.insert(0, lib_decomren)

import itertools
import numpy as np
import torch
import decompress_render

In [39]:
import numpy as np

def lookat_to_colmap_quaternion(eye, lookat, up=np.array([0.0, -1.0, 0.0])):
    """
    Convert eye + lookat + up → COLMAP quaternion (w, x, y, z) and translation (t_x, t_y, t_z)
    
    Parameters
    ----------
    eye     : array-like (3,)   → camera position in world (e.g. [2, 3, 4])
    lookat  : array-like (3,)   → point the camera is looking at
    up      : array-like (3,)   → up direction (default = OpenGL/Blender style: Y down)
              Use [0,1,0] if your world has +Y up (like most COLMAP datasets)

    Returns
    -------
    qvec    : np.array (4,)     → [qw, qx, qy, qz]  (COLMAP order, unit quaternion)
    tvec    : np.array (3,)     → translation that COLMAP expects (world → camera)
    R       : np.array (3,3)    → rotation matrix (world → camera)  [optional bonus]
    """
    eye    = np.asarray(eye,    dtype=np.float64)
    lookat = np.asarray(lookat, dtype=np.float64)
    up     = np.asarray(up,     dtype=np.float64)

    # Forward vector (from camera to target)
    forward = lookat - eye
    forward = forward / np.linalg.norm(forward)

    # Right vector
    right = np.cross(forward, up)
    right = right / np.linalg.norm(right)

    # Recompute up to be orthogonal (Gram-Schmidt)
    up_clean = np.cross(right, forward)   # note: forward × right would be wrong order

    # Build rotation matrix (world → camera)
    # Each column is a basis vector of the camera frame expressed in world
    R = np.stack((right, -up_clean, -forward), axis=1)   # note the signs!
    # Explanation of signs:
    #   +Xcam → right
    #   +Ycam → down (if up was [0,-1,0]) → we want -up_clean
    #   +Zcam → backward (opposite of forward)

    # Convert rotation matrix → quaternion (scalar-first!)
    # Using the most numerically stable method
    trace = R[0,0] + R[1,1] + R[2,2]
    
    if trace > 0:
        s = 0.5 / np.sqrt(trace + 1.0)
        w = 0.25 / s
        x = (R[2,1] - R[1,2]) * s
        y = (R[0,2] - R[2,0]) * s
        z = (R[1,0] - R[0,1]) * s
    else:
        if R[0,0] > R[1,1] and R[0,0] > R[2,2]:
            s = 2.0 * np.sqrt(1.0 + R[0,0] - R[1,1] - R[2,2])
            w = (R[2,1] - R[1,2]) / s
            x = 0.25 * s
            y = (R[0,1] + R[1,0]) / s
            z = (R[0,2] + R[2,0]) / s
        elif R[1,1] > R[2,2]:
            s = 2.0 * np.sqrt(1.0 + R[1,1] - R[0,0] - R[2,2])
            w = (R[0,2] - R[2,0]) / s
            x = (R[0,1] + R[1,0]) / s
            y = 0.25 * s
            z = (R[1,2] + R[2,1]) / s
        else:
            s = 2.0 * np.sqrt(1.0 + R[2,2] - R[0,0] - R[1,1])
            w = (R[1,0] - R[0,1]) / s
            x = (R[0,2] + R[2,0]) / s
            y = (R[1,2] + R[2,1]) / s
            z = 0.25 * s

    qvec = np.array([w, x, y, z])
    
    # COLMAP translation (world → camera): t = -R @ eye
    tvec = -R @ eye

    return qvec, tvec, R

In [40]:
def qvec2rotmat(qvec):
    """Convert COLMAP qvec (w, x, y, z) to 3x3 rotation matrix"""
    w, x, y, z = qvec
    return np.array([
        [1 - 2*y*y - 2*z*z,  2*x*y - 2*z*w,      2*x*z + 2*y*w],
        [2*x*y + 2*z*w,      1 - 2*x*x - 2*z*z,  2*y*z - 2*x*w],
        [2*x*z - 2*y*w,      2*y*z + 2*x*w,      1 - 2*x*x - 2*y*y]
    ])

def colmap_pose_to_camera_to_world(qvec, tvec):
    """
    Input: qvec (4,), tvec (3,) from images.txt
    Output: 4x4 camera-to-world matrix (numpy array)
    """
    R = qvec2rotmat(qvec)           # world → cam rotation
    t = tvec.reshape(3, 1)           # world → cam translation
    
    # Camera-to-world = [R^T | -R^T t]
    cam_to_world = np.eye(4)
    cam_to_world[:3, :3] = R.T
    cam_to_world[:3, [3]]   = -R.T @ t
    return cam_to_world

In [147]:
# Example 1: Camera looking down -Z axis (common in many datasets)
eye    = [0.0, 0.0, 0.0]
lookat = [1.0, 1.0, 1.0]
up     = [0.0, -1.0, 0.0]      # +Y up (standard COLMAP convention)

eye    = [2.0, -3.0, 4.0]
lookat = [0.0, 0.0, 0.0]
up     = [0.0, 0.0, 1.0]

q, t, R = lookat_to_colmap_quaternion(eye, lookat, up)
print("qvec:", q.round(6))
print("tvec:", t.round(6))


cam2world = colmap_pose_to_camera_to_world(q, t)
cam2world

qvec: [ 0.699428 -0.040193  0.132748  0.050999]
tvec: [-1.9136   -0.735151 -4.979728]


array([[ 0.95955428,  0.06066858, -0.18979489,  0.93567716],
       [-0.08201065,  0.99156737, -0.04268413,  0.35946046],
       [ 0.18159579,  0.06976392,  0.96152505,  5.18692157],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [141]:
cam2world = colmap_pose_to_camera_to_world(
    np.array([0.92387953, 0.38268343/np.sqrt(2),  0.0, 0.38268343/np.sqrt(2)]), 
    np.array([0.0, 0.0, 0.0])
)
cam2world




array([[ 0.85355339,  0.5       ,  0.14644661,  0.        ],
       [-0.5       ,  0.70710678,  0.5       ,  0.        ],
       [ 0.14644661, -0.5       ,  0.85355339,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [126]:

nlevel = torch.tensor([2], dtype=torch.int8).cuda()
vox_roots = torch.tensor(tuple(itertools.product([1, 2], [1, 2], [1, 2])), dtype=torch.float32).cuda() * 2**(-2)
vox_len = torch.ldexp(torch.tensor([1], dtype=torch.int8).cuda(), -nlevel)
vox_roots, vox_len

(tensor([[0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.5000],
         [0.2500, 0.5000, 0.2500],
         [0.2500, 0.5000, 0.5000],
         [0.5000, 0.2500, 0.2500],
         [0.5000, 0.2500, 0.5000],
         [0.5000, 0.5000, 0.2500],
         [0.5000, 0.5000, 0.5000]], device='cuda:0'),
 tensor([0.2500], device='cuda:0'))

In [134]:

camera_settings = decompress_render.renderer.CameraSettings(
    image_width=256, 
    image_height=256, 
    tanfovx=1.0, 
    tanfovy=1.0, 
    cx=128.0, 
    cy=128.0, 
    w2c_matrix=torch.tensor([cam2world], dtype=torch.float32).clone().cuda().inverse(),
    c2w_matrix=torch.tensor([cam2world], dtype=torch.float32).clone().cuda()
)
render_settings = decompress_render.renderer.RenderSettings(
    num_sample_per_vox=3,
    bg_color=3,
    near=0.001,
    need_depth=False,
    track_max_w=False,
    debug=True,
)

In [135]:
camera_settings.c2w_matrix, camera_settings.w2c_matrix

(tensor([[[ 0.8536,  0.5000,  0.1464,  0.0000],
          [-0.5000,  0.7071,  0.5000,  0.0000],
          [ 0.1464, -0.5000,  0.8536,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0'),
 tensor([[[ 0.8536, -0.5000,  0.1464,  0.0000],
          [ 0.5000,  0.7071, -0.5000,  0.0000],
          [ 0.1464,  0.5000,  0.8536,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0'))

In [136]:
temp = (camera_settings.w2c_matrix[0, :3, :3, ] @ vox_roots.T).T
temp, temp[:, :2]/temp[:, [2]]

(tensor([[ 1.2500e-01,  1.7678e-01,  3.7500e-01],
         [ 1.6161e-01,  5.1777e-02,  5.8839e-01],
         [-1.4901e-08,  3.5355e-01,  5.0000e-01],
         [ 3.6612e-02,  2.2855e-01,  7.1339e-01],
         [ 3.3839e-01,  3.0178e-01,  4.1161e-01],
         [ 3.7500e-01,  1.7678e-01,  6.2500e-01],
         [ 2.1339e-01,  4.7855e-01,  5.3661e-01],
         [ 2.5000e-01,  3.5355e-01,  7.5000e-01]], device='cuda:0'),
 tensor([[ 3.3333e-01,  4.7140e-01],
         [ 2.7467e-01,  8.7998e-02],
         [-2.9802e-08,  7.0711e-01],
         [ 5.1321e-02,  3.2038e-01],
         [ 8.2211e-01,  7.3316e-01],
         [ 6.0000e-01,  2.8284e-01],
         [ 3.9766e-01,  8.9181e-01],
         [ 3.3333e-01,  4.7140e-01]], device='cuda:0'))

In [137]:
a, b, c = decompress_render.renderer.rasterize_voxels(
    camera_settings, render_settings,
    vox_roots,
    vox_len,
    lambda x: None
)
a, b, c

N: 8 
chunk_size: 1407 


(tensor([64, 25, 36, 20, 36, 30, 25, 20], device='cuda:0', dtype=torch.int32),
 tensor([64,  0,  0,  ...,  0,  0,  0], device='cuda:0', dtype=torch.uint8),
 tensor([127.5000, 138.7637, 232.7295, 241.6512, 134.0691, 115.8102, 204.3000,
         187.8398, 101.9000, 168.5083, 178.4003, 254.2847, 114.0054, 141.8021,
         170.1667, 205.0797, 170.1667, 163.7039, 256.0000, 256.0000, 165.5805,
         135.4050, 241.3337, 205.3718, 144.6002, 187.8398, 222.7986, 256.0000,
         145.7857, 157.8666, 202.9041, 219.5153], device='cuda:0'))

In [138]:
a

tensor([64, 25, 36, 20, 36, 30, 25, 20], device='cuda:0', dtype=torch.int32)

In [139]:
vox_roots, c.reshape((8, 2, 2)).int()

(tensor([[0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.5000],
         [0.2500, 0.5000, 0.2500],
         [0.2500, 0.5000, 0.5000],
         [0.5000, 0.2500, 0.2500],
         [0.5000, 0.2500, 0.5000],
         [0.5000, 0.5000, 0.2500],
         [0.5000, 0.5000, 0.5000]], device='cuda:0'),
 tensor([[[127, 138],
          [232, 241]],
 
         [[134, 115],
          [204, 187]],
 
         [[101, 168],
          [178, 254]],
 
         [[114, 141],
          [170, 205]],
 
         [[170, 163],
          [256, 256]],
 
         [[165, 135],
          [241, 205]],
 
         [[144, 187],
          [222, 256]],
 
         [[145, 157],
          [202, 219]]], device='cuda:0', dtype=torch.int32))

In [ ]:
cam_conner = c.reshape((64, 3))
cam_conner_scaled = cam_conner[:, :2] / cam_conner[:, [2]]
cam_conner_scaled = cam_conner_scaled.reshape((8, 8, 2))
cam_conner_scaled

tensor([[[-0.0000, -0.0000],
         [-0.3062,  0.1768],
         [-0.0000, -0.3536],
         [-0.2449, -0.1414],
         [ 0.3062,  0.1768],
         [-0.0000,  0.2828],
         [ 0.2449, -0.1414],
         [-0.0000, -0.0000]],

        [[-0.3062,  0.1768],
         [-0.4899,  0.2828],
         [-0.2449, -0.1414],
         [-0.4082, -0.0000],
         [-0.0000,  0.2828],
         [-0.2041,  0.3536],
         [-0.0000, -0.0000],
         [-0.1750,  0.1010]],

        [[-0.0000, -0.3536],
         [-0.2449, -0.1414],
         [-0.0000, -0.5657],
         [-0.2041, -0.3536],
         [ 0.2449, -0.1414],
         [-0.0000, -0.0000],
         [ 0.2041, -0.3536],
         [-0.0000, -0.2020]],

        [[-0.2449, -0.1414],
         [-0.4082, -0.0000],
         [-0.2041, -0.3536],
         [-0.3499, -0.2020],
         [-0.0000, -0.0000],
         [-0.1750,  0.1010],
         [-0.0000, -0.2020],
         [-0.1531, -0.0884]],

        [[ 0.3062,  0.1768],
         [-0.0000,  0.2828],
      

In [33]:
vox_roots, cam_conner_scaled.min(axis=1)[0], cam_conner_scaled.max(axis=1)[0]

(tensor([[0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.5000],
         [0.2500, 0.5000, 0.2500],
         [0.2500, 0.5000, 0.5000],
         [0.5000, 0.2500, 0.2500],
         [0.5000, 0.2500, 0.5000],
         [0.5000, 0.5000, 0.2500],
         [0.5000, 0.5000, 0.5000]], device='cuda:0'),
 tensor([[-0.3062, -0.3536],
         [-0.4899, -0.1414],
         [-0.2449, -0.5657],
         [-0.4082, -0.3536],
         [-0.0000, -0.1414],
         [-0.2041, -0.0000],
         [-0.0000, -0.3536],
         [-0.1750, -0.2020]], device='cuda:0'),
 tensor([[0.3062, 0.2828],
         [-0.0000, 0.3536],
         [0.2449, -0.0000],
         [-0.0000, 0.1010],
         [0.4899, 0.3536],
         [0.2041, 0.4041],
         [0.4082, 0.1010],
         [0.1750, 0.1768]], device='cuda:0'))

In [26]:
cam_conner_scaled.shape

torch.Size([8, 8, 2])

In [ ]:
# # SH colors
# SH_C0 = 0.28209479177387814
# SH_C1 = 0.4886025119029199
# SH_C2 = [ 1.0925, -1.0925, 0.3153, -1.0925, 0.5462 ]
# SH_C3= [ -0.5900, 2.8906, -0.4570, 0.3731, -0.4570, 1.4453, -0.5900]

# # 
# def computeColorFromSH( vox_center, cam_center, sh0, sh, degree=3):
#     # vox_center, cam_center: array of 3D position - [x, y, z]
#     # sh0: [cR, cG, cB]
#     # sh: array (num_SH_coefs-1) x 3
    
#     # Normalized ray direction
#     dir = vox_center - cam_center
#     dir = dir * 1 / (dir[0]**2, dir[1]**2, dir[2]**2)

#     # Result is vector 3: [R, G, B]
#     result = SH_C0 * sh0

#     if (degree > 0):
#         x = dir[0]
#         y = dir[1]
#         z = dir[2]
#         result = result - SH_C1 * y * sh[0] + SH_C1 * z * sh[1] - SH_C1 * x * sh[2]

#         if (degree > 1):
#             xx = x * x
#             yy = y * y
#             zz = z * z
#             xy = x * y
#             yz = y * z
#             xz = x * z

#             result = result + \
#                 SH_C2[0] * xy * sh[3] + \
#                 SH_C2[1] * yz * sh[4] + \
#                 SH_C2[2] * (2.0 * zz - xx - yy) * sh[5] + \
#                 SH_C2[3] * xz * sh[6] + \
#                 SH_C2[4] * (xx - yy) * sh[7]

#             if (degree > 2):
#                 result = result + \
#                     SH_C3[0] * y * (3.0 * xx - yy) * sh[8] + \
#                     SH_C3[1] * xy * z * sh[9] + \
#                     SH_C3[2] * y * (4.0 * zz - xx - yy) * sh[10] + \
#                     SH_C3[3] * z * (2.0 * zz - 3.0 * xx - 3.0 * yy) * sh[11] + \
#                     SH_C3[4] * x * (4.0 * zz - xx - yy) * sh[12] + \
#                     SH_C3[5] * z * (xx - yy) * sh[13] + \
#                     SH_C3[6] * x * (xx - 3.0 * yy) * sh[14]
        
#     result = result + 0.5

#     # RGB colors are clamped to non-negative values.
#     result[0] *= (result[0] > 0.0)
#     result[1] *= (result[1] > 0.0)
#     result[2] *= (result[2] > 0.0)
#     return result
